In [51]:
import pandas as pd
import json
import pickle
import os
import itertools
from ast import literal_eval
import numpy as np
import sys
from datetime import datetime
from IPython.display import clear_output, display
import subprocess
from datetime import datetime
pd.options.display.float_format = '{:.0f}'.format

In [52]:
def LoadJsonFile(filename): 
    with open(filename, 'r') as f:
        DicConfig = json.load(f)
    return DicConfig


def GlobalDicDeplier(OneDic):
    for k,v in OneDic.items():
        exec('globals()[k] = v')
    return None

In [53]:
DicConfig = LoadJsonFile(os.path.join(os.getcwd(),"config.json"))
GlobalDicDeplier(DicConfig)
sys.path.append(Root)
from fun import *

print("Load Config variables")

Load Config variables


In [54]:
path = os.path.join(Root,FolderProject,"RefFam.pkl")
RefFam = LoadPickleOrInit(path)

path = os.path.join(Root,FolderProject,"RefRT.pkl")
RefRT = LoadPickleOrInit(path)

tweetsdf = RefFam.copy()[RefFam.status=="ok"]
rtdf = RefRT.copy()[RefRT.status=="ok"]

datefirst = rtdf.TWEETUNIXEPOCH.min()
datelast = rtdf.TWEETUNIXEPOCH.max()

### Define Time Marks

Pour une time mark, on doit sortir deux périodes:
 * La fenetre graph (window size)
 * La fenêtre tweets (step size)

In [179]:
# Build LinksDF from time mark
randomtm = TimeMarksList[-1]
LinksDic = BuildLinksDocFromTimeMark(randomtm,WindowSize,StepSize,rtdf)

In [169]:
# Build Auteur -> tweetsid df from rtdf

In [ ]:
def BuildLinksDocFromTimeMark(randomtm,WindowSize,StepSize,rtdf):
    periods = ExtractTheTwoPeriods(randomtm,WindowSize,StepSize)
    debut,fin = periods["GraphWindow"]
    myfilter = (rtdf.TWEETUNIXEPOCH>=debut) & (rtdf.TWEETUNIXEPOCH<=fin)
    temprtdf = rtdf.copy()[myfilter]
    temprtdf.reset_index(drop=True,inplace=True)
    list_of_authors = temprtdf.groupby('USERID')['AUTHORID'].apply(list)
    LinksDic = GetLinksFromPeriod(list_of_authors)
    return LinksDic

In [175]:
def GetLinksFromPeriod(list_of_authors):
    
    L = []
    for SetOfIds in list_of_authors:
        if len(SetOfIds)>1:
            L = L + ExtractCleanLinksFromSets(SetOfIds)
    
    linkscompteur = {}
    for item in L:
        linkscompteur[item[0]] = linkscompteur.get(item[0],0) + item[1]
    
    return linkscompteur

In [163]:
def ExtractCleanLinksFromSets(SetOfIds):
    SetOfIds = [int(item) for item in SetOfIds]
    RawLinks = ExtractRawLinks(SetOfIds)
    CleanLinks = [ExplicitLink(item) for item in RawLinks]
    return CleanLinks

In [138]:
def ExtractRawLinks(SetOfIds):
    
    x = np.array(SetOfIds)
    comptage = np.unique(x,return_counts=True)
    list_of_tuples = np.vstack(comptage).T.tolist()
    combinations = list(itertools.combinations(list_of_tuples,2))
    
    return combinations

In [153]:
def ExplicitLink(x):
    weight = min([item[1] for item in x])
    link = tuple(item[0] for item in x)
    return link,weight

In [ ]:
def DefineTimeMarks(lastdate,firstdate,step,window):

    mark0 = firstdate + window
    start = mark0
    L = [start]

    while(True):
        newmark = start+step
        if newmark>lastdate:
            break
        else:
            L.append(newmark)
            start=newmark

    return L

In [ ]:
def ExtractTheTwoPeriods(timemark,WindowSize,StepSize):
    res = {"GraphWindow":(timemark-WindowSize,timemark),
           "TweetWindow":(timemark-StepSize,timemark)}
    return res